# Automated finite difference operators from symbolic equations

This notebook is the first in a series of hands-on tutorial notebooks that are intended to give a brief practical overview of the [Devito](http://www.opesci.org/devito-public) finite difference framework. We will present an overview of the symbolic layers of Devito and how to use them to define a set of simple `Operator` objects.

In [ ]:
from devito import *
from sympy import init_printing, symbols, solve
init_printing(use_latex=True)

## From equation to stencil code in a few lines of Python

Today's objective is to demonstrate the 

## Basic concepts

At the very core of Devito are two types of objects:
- **Symbolic expressions** that define stencil computations as a set of `sympy.Equation` objects
- **Operator** objects that transform that auto-generate and execute optimized C code from these high-level expressions

The symbolic expressions needed to define the computational operators are based on Devito's `Function` objects. A `Function` object does two things:
1. It behaves like a `sympy.Function` symbol
2. It carries around user data

To define manage the data associated with our symbol we need to know some information, like the shape and phsyical extent (size in units) of our computational domain. We can define this using a `Grid` object, from which to create our `Function`. Depending on the shape of the computational grid, Devito will create a set of `Dimension` objects that also behave like SymPy symbols. 

In [ ]:
?Function

In [ ]:
grid = Grid(shape=(5, 6), extent=(1., 1.))
x, y = grid.dimensions
f = Function(name='f', grid=grid)

## Symbolic behaviour of Function objects

Let's have a look at the symbolic behaviour first by taking a derivative in `x`. For this we first create a derivative symbol and then we evaluate it as a finite difference expression.

In [ ]:
f

In [ ]:
f.diff(x)

Using SymPy we can now discretize this derivative symbol using the finite difference method. First, we let SymPy create a generic central difference expression.

In [ ]:
f.diff(x).as_finite_difference()

Next we can tell SymPy to insert the symbol $h_x$ to represent the spacing of our grid in the $x$ dimension. SymPy will still use a central differencing scheme, where we ae required to know the values of $f(x, y)$ at distances $h_x / 2$.

In [ ]:
h_x = symbols('h_x')
f.diff(x).as_finite_difference(h_x)

To avoid values half-cell offsets we can force SymPy to use a forward differencing scheme by providing the explicit stencil points for which to discretize our derivative. This stencil expression now aligns with our computational grid.

In [ ]:
f.diff(x).as_finite_difference([x, x+h_x])

Typing out this expression every time I want to take a simple derivative is somewhat tedious. Luckily, Devito symbols provide a set of shorthand notations to generate the most common derivatives automatically. The first derivative in x, for example, is simply: 

In [ ]:
f.dx

In fact, we are slightly cheating here, since we actually used the forward difference scheme, or "right derivtive", not the central one. If we wanted to be really explicit, we could use the shorthand `f.dxr` to create the same expression.

In [ ]:
f.dxr

However, following the same convention we can also use the shorthand `f.dxl` to create the "left derivative", or in other words use a backward differenceing scheme. We will need the backward difference symbol in our worked example later.

In [ ]:
f.dxl

## Time-dependent functions and data

By default Devito's `Function` objects will use the spatial dimensions `(x, y)` for 2D grids and `(x, y, z)` for 3D grids. To solve a PDE over several timesteps though, we need a time dimension for our symbolic function. For this Devito provides a second function type, `TimeFunction` that provides the correct dimension, and some other intricacies needed to create a time stepping scheme.

In [ ]:
g = TimeFunction(name='g', grid=grid)
g

We can take the same spatial derivatives of $g$ as before and we can now also take derivative wrt. $t$.

In [ ]:
g.dx

In [ ]:
g.dt

The careful observer might have recognised that the spacing symbol for $t$ is called $dt$. This is due to the special status of the time dimension and will come in handy once we start building operators.

#### Data associated with functions

The other "special" property of `TimeFunction` objects is that they have a slightly different data layout due to the fact that we assume $t$ to be the stepping dimension in our time stepping scheme. As we said in the beginning, Devito functions do not only behave symbolically like SymPy functions, but they also hold and manage user data. So, let's first take a look at our original function $f$.

In [ ]:
f.data

In [ ]:
f.data.shape

The `.data` property of each function presents the data that Devito has allocated to the user as a Numpy array. The fact that the function object allocates the user data ensures that Devito can manage data layout internally, which is of vital importance for some advanced performance optimisation techniques.

As we can see, the allocated data is of `shape=(5, 6)`, the shape that we previously defined on the Grid. Now look at the shape of the data associated with $g$.

In [ ]:
g.data.shape

The `TimeFunction` has allocated two buffers of shape `(5, 6)`, allowing us to perform a stencil computation with alternating buffers. These correspond symbolically to $g(t, x, y)$ and $g(t + dx, x, y)$.

## A few words on the Operator

The other important class provided by Devito is the `Operator`. An `Operator` takes as input one or more SymPy expressions in the form of a `sympy.Eq` object that expresses an assignment to a function. The dimension indices used in these functions are interpreted as loops over the respective iteration space, eg. assignint a value to $f(x, y)$ assigns the value to all points in the $x$, $y$ space. 

In [ ]:
Operator(Eq(f, 2.))()
f.data

## A simple operator: Linear convenction

**Note:** The following example is derived from [step 5](http://nbviewer.ipython.org/github/barbagroup/CFDPython/blob/master/lessons/07_Step_5.ipynb) of the tutorials in the excellent tutorial series [CFD Python: 12 steps to Navier-Stokes](http://lorenabarba.com/blog/cfd-python-12-steps-to-navier-stokes/).

In this simple example we will show how to derive a very simple convection operator from a high-level descritpion of the governing equation. We will go through the process of deriving a discretized finite difference formulation of the state update for the field variable $u$, before creating a callable `Operator` object. Luckily, the automation provided by SymPy makes the derivation very nice and easy.

The governing equation we want to implement is the linear convection equation:
$$\frac{\partial u}{\partial t}+c\frac{\partial u}{\partial x} + c\frac{\partial u}{\partial y} = 0$$

In [ ]:
from examples.cfd import init_hat, plot_field

# First we define some parameters
nx, ny = (81, 81)  # Number of grid point in x and y
nt = 100  # Number of timesteps
dx = 2. / (nx - 1)  # Grid spacing in x and y
dy = 2. / (ny - 1)
dt = 0.2 * dx  # Timestep size (sigma=0.2)
c = 1  # Value for c

# Then we create a grid and our function
grid = Grid(shape=(nx, ny), extent=(2., 2.))
x, y = grid.dimensions
u = TimeFunction(name='u', grid=grid)

# We can now set the initial condition and plot it
init_hat(field=u.data[0], dx=dx, dy=dy, value=2.)
init_hat(field=u.data[1], dx=dx, dy=dy, value=2.)

plot_field(u.data[0])

Next, we want to discretize our governing equation so that we can create a functional `Operator` from it. We can start by simply writing out the equation as a symbolic expression, while using the shorthand expressions for derivatives that the `Function` objects provide. This will create a symbolic object of the dicrestized equation.

In [ ]:
eq = Eq(u.dt + c * u.dxl + c * u.dyl)
eq

As we can see, SymPy as kindly resolved our derivatives. Next, we need to rearrange our equation so that the term $u(t+dt, x, y)$ is on the left-hand side, since it represents the next point in time for our state variable $u$. We can again use SymPy to rearrange our equation for us, so that it represents a valid state update for $u$.

In [ ]:
stencil = solve(eq, u.forward)[0]
update = Eq(u.forward, stencil)
update

From this update epxression, we can now create an `Operator`. This `Operator` will basically behave like a Python function that we can call, as long as we provide all necessary unknowns. In this case we need to provide the number of timesteps to compute via the keyword `time` and the timestep size to use via `dt`.

In [ ]:
op = Operator(update)
op(time=nt+1, dt=dt)

plot_field(u.data[0])

## Exercise 1: A coupled system

The solution of the previous example showed a lot of numerical diffusion. This is somewhat expected as the discretization we have chosen is not very good at capturing sharp gradients, like the edge of our "hat". A better formulation of the equation is given by the coupled system of equations:

\begin{aligned}
\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} = 0 \\
\\
\frac{\partial v}{\partial t} + u \frac{\partial v}{\partial x} + v \frac{\partial v}{\partial y} = 0\\
\end{aligned}

To implement a coupled set of equations, all we have to do is create update expressions for both state variables and hand both expressions to the `Operator`. This is simply done by concatenating them in a Python list. To create our symbolic derivatives we want to use the same backward differencing scheme (_left derivative_) as in the previous example.

Below is the setup of the initial state. Note that we now have two state variables.

In [ ]:
grid = Grid(shape=(nx, ny), extent=(2., 2.))
u = TimeFunction(name='u', grid=grid)
v = TimeFunction(name='v', grid=grid)

# We use the same initial condition and plot it
init_hat(field=u.data[0], dx=dx, dy=dy, value=2.)
init_hat(field=u.data[1], dx=dx, dy=dy, value=2.)
init_hat(field=v.data[0], dx=dx, dy=dy, value=2.)
init_hat(field=v.data[1], dx=dx, dy=dy, value=2.)

plot_field(u.data[0])

In [ ]:
# Please write your solution here.

We can look at the final result again. We should see a similar displacement in our "hat", but this time the resulting shape should be slightly skewed towards the direction of travel. This is still not a great result, but that is expeted, given the simplicity of the numerical scheme we're using.

In [ ]:
plot_field(u.data[0])

<button data-toggle="collapse" data-target="#sol1" class='btn btn-primary'>Solution</button>
<div id="sol1" class="collapse">
```python
eq_u = Eq(u.dt + u*u.dxl + v*u.dyl)
eq_v = Eq(v.dt + u*v.dxl + v*v.dyl)

stencil_u = solve(eq_u, u.forward)[0]
stencil_v = solve(eq_v, v.forward)[0]
update_u = Eq(u.forward, stencil_u)
update_v = Eq(v.forward, stencil_v)

op = Operator([update_u, update_v])
op(time=nt+1, dt=dt)
```

## Second derivatives and high-order stencils

For the above example all we had to do was combine some first derivatives. However, lots of common scientific problems require second derivative, most notably any PDE including diffusion. To generate second order derivatives we need to give the `devito.Function` object another piece of information: the desired discretization of the stencils. 

First, let's do a simple second derivative in $x$, for which we need to give $u$ at least a `space_order` of `2`. The shorthand for the second derivative is then `u.dx2`. 

In [ ]:
u = TimeFunction(name='u', grid=grid, space_order=2)
u.dx2

We can arbitrarily drive the discretization order up if require higher order stencils. 

In [ ]:
u = TimeFunction(name='u', grid=grid, space_order=4)
u.dx2

To implement a diffusion operator, we need to take the Laplacian $\nabla^2 u$, which is simply the second derivative in all space dimensions. For this, Devito also provides a shorthand expression, which means I do not have to hardcode the problem dimension (2D or 3D) in my code. I simply give another shape to the grid and let it figure our the Laplacian like this:

In [ ]:
grid_3d = Grid(shape=(5, 6, 7), extent=(1., 1., 1.))
u = TimeFunction(name='u', grid=grid_3d, space_order=2)
u.laplace

Just for kicks (and to show off the power of SymPy), let's see what the 12th-order Laplacian of a three dimensional time-dependent function looks like.

In [ ]:
grid_3d = Grid(shape=(5, 6, 7), extent=(1., 1., 1.))
u = TimeFunction(name='u', grid=grid_3d, space_order=12)
u.laplace

## Exercise 2: Making a wave

In the final exercise of the introduction we will implement a simple wave equation operator to the ones used in seismic imaging. For this we will implement the isotropic wave equation without boundary conditions. The equation defines the propagation of a wave in an isotropic medium and is defined as

$$m \frac{\partial^2 u}{\partial t^2} = \nabla^2 u$$

where $m$ is the square slowness of the wave, defined in terms of the wave speed $c$ as $m = 1 / c^2$. For the purpose of this exercise, we will ignore any source terms and instead use a "warmed-up" wavefield from file.

In the cell below we define the time parameters of our simulation, as well as the spatial dimensions and the shape of our computational grid with a `Grid` object. Using this grid object we can define two functions:
* The wavefield $u(t, x, y)$ which we initialise from the file `wavefield.npy`
* The square slowness $m(x, y)$ which, for now we will keep constant, for $c = 1.5km/s$.

In [ ]:
import numpy as np
from examples.seismic import plot_image

t0, tn, dt = 214., 400, 4.2  # Start, end and timestep size
nt = int(1 + (tn - t0) / dt)  # Number of timesteps

# A 120x120 grid that defines our square domain
grid = Grid(shape=(120, 120), extent=(1800., 1800.))

# Load and plot the initial "warmed-up" wavefield
u = TimeFunction(name='u', grid=grid, space_order=2, time_order=2)
u.data[:] = np.load('wavefield.npy')
plot_image(u.data[0])

# Square slowness for a constant wave speed of 1.5m/s
m = Function(name='m', grid=grid)
m.data[:] = 1. / 1.5**2

To remind ourselves, the governing equation we want to implement is

$$m \frac{\partial^2 u}{\partial t^2} = \nabla^2 u$$

Please have a go and try to implement the operator below. You will need to follow the same strategy to discretize the equation and create a symbolic stencil expression that updates `u(t + dt, x, y)$. Once we apply our `Operator` for `nt` timesteps we should see that the wave has expanded homogeneously.

In [ ]:
# Reset the wavefield, so that we can run the cell  multiple times
u.data[:] = np.load('wavefield.npy')

# Please implement your wave equation operator here 

In [ ]:
plot_image(u.data[0])

<button data-toggle="collapse" data-target="#sol2" class='btn btn-primary'>Solution</button>
<div id="sol2" class="collapse">
```python
eqn = Eq(m * u.dt2 - u.laplace)
stencil = solve(eqn, u.forward)[0]
update = Eq(u.forward, stencil)
op = Operator(update)

op(t=nt, dt=dt)

```

Now, let's see what happens if we change the square slowness field `m` by increasing the wave speed to $2.5$ in the bottom half of the domain.

In [ ]:
m.data[:, 60:] = 1. / 2.5**2  # Set a new wave speed
plot_image(m.data)

u.data[:] = np.load('wavefield.npy')  # Reset our wave field u
plot_image(u.data[0])

op(t=60, dt=dt)
plot_image(u.data[0])

<sup>This notebook is part of the tutorial "Optimised Symbolic Finite Difference Computation with Devito" presented at the Intel® HPC Developer Conference 2017.</sup>